In [1]:
##### WARNING: SHOULD BE IGNORED
# USE setup_data.py

# type: ignore

from PIL import Image
import requests
import numpy as np
import pandas as pd 
import geopandas as gpd
import pyproj
from shapely.ops import transform
import shapely
from shapely import Point
import os
import django

env='dev'
pd.set_option('display.precision', 2)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", f"water.settings.{env}")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from water import models as m
from django.db.models import Count
from water.utils import parse as p

In [25]:
from water.utils import data

m.ReservoirGeo.objects.all()

<QuerySet []>

In [2]:
filename_all = p.filename_csv_all
df_all_raw = pd.read_csv(filename_all).sort_values(['province', 'reservoir', 'ds'])
name_df = df_all_raw.reservoir.unique()

file = f'{p.folder_data}/raw/reservoirs/InfGeografica/InfVectorial/Gpkg/Inv_presas_embalses.gpkg'

# Open the file in geopandas
gdf_raw = gpd.read_file(file)

name_geo = gdf_raw[gdf_raw.nombre.notnull()].nombre.unique()

replace_vals = ['embalse del ', 'embalse de ', 'embalse ']

def clean_name(name):
    for val in replace_vals:
        name = name.lower().replace(val, '')
    return name

names_geo_dict = {name: clean_name(name) for name in name_geo}

import difflib

diff_obs = difflib.ndiff('ab', 'cd')

# Summarize the differences
# Show number of changes
def diffnum_difflib(a,b):
    diff = difflib.ndiff(a, b)
    diff = list(diff)
    diff = [x for x in diff if x[0] != ' ']
    return len(diff)

df_matches = pd.DataFrame(columns=['name_df', 'name_geo', 'diff'])
df_matches['name_df'] = name_df
geo_clean = list(names_geo_dict.values())

for i, name in enumerate(name_df):
    diffs = []
    for j, name2 in enumerate(geo_clean):
        diffs.append(diffnum_difflib(name, name2))
    df_matches['diff'].iloc[i] = min(diffs)
    df_matches['name_geo'].iloc[i] = geo_clean[diffs.index(min(diffs))]

df_matches['name_geo_full'] = df_matches['name_geo'].map({v: k for k, v in names_geo_dict.items()})
df_matches.sample(10)

dict_matches = df_matches.set_index('name_geo_full')['name_df'].to_dict()

capacities = df_all_raw.groupby(['province', 'reservoir'])['capacity_hm3'].agg(['last', 'nunique']).reset_index()
df_all_raw.query('reservoir=="el limonero"').capacity_hm3.unique()


array([24.7, 22.3])

In [8]:
ds_start = "2012-09-01"
province = "malaga"
num_states = 10000
num_res_max = 5

capacities = df_all_raw.groupby(['province', 'reservoir'])['capacity_hm3'].agg(['last', 'nunique']).reset_index()
capacities = capacities[capacities.province == province].copy()
capacities = capacities.sort_values('last', ascending=False).head(num_res_max)
capacities

df_selected = df_all_raw[df_all_raw.reservoir.isin(capacities.reservoir.unique())]
df_selected = df_selected[df_selected.ds >= ds_start].copy()

name_to_full = {v: k for (k, v) in dict_matches.items()}

In [9]:
m.ReservoirState.objects.all().delete()
m.Reservoir.objects.all().delete()

for _, row in capacities.iterrows():
    name=row['reservoir']
    province = row['province']
    name_full = name_to_full[name]
    reservoir = m.Reservoir.objects.create(name=row['reservoir'], name_full=name_full, province=province, capacity=row['last'])
    reservoir.save()
    
print(len(df_selected))
    
for _, row in df_selected.head(num_states).iterrows():
    reservoir = m.Reservoir.objects.get(name=row['reservoir'])
    state = m.ReservoirState.objects.create(
        reservoir=reservoir,
        date=row['ds'],
        volume=row['stored_hm3'],
    )
    state.save()
    
m.ReservoirState.objects.all().count()

4406


4406

In [6]:
df_all = p.add_cols(df_selected)
df_all['date_lag'] = df_all.groupby(['province', 'reservoir'])['date'].shift(1)
df_all['date_diff'] = (df_all.date - df_all.date_lag).dt.days

cols = ['rainfallsince', 'stored_hm3', 'capacity_hm3']
for var in ['rainfallsince', 'stored_hm3']:
    df_all[f'{var}_diff'] = df_all.groupby(['province', 'reservoir'])[var].diff()
    df_all[f'{var}_diff_0'] = df_all[f'{var}_diff']
    for lags in range(1, 10):
        df_all[f'{var}_diff_{lags}'] = df_all.groupby(['province', 'reservoir'])[f'{var}_diff'].shift(lags)

# Delete all rainfall objects
m.RainFall.objects.all().delete()

# Get all reservoirs with state data
reservoirs = m.Reservoir.objects.annotate(
        num_states=Count("reservoir_reservoirstate")
    ).filter(num_states__gt=0)

reservoir_names = reservoirs.values_list('name', flat=True)

for _, row in df_all[df_all.reservoir.isin(reservoir_names)].head(num_states).iterrows():
    reservoir = m.Reservoir.objects.get(name=row['reservoir'])
    rainfall = m.RainFall.objects.create(
        date=row['ds'],
        reservoir=reservoir,
        amount=row['rainfallsince_diff'],
        amount_cumulative=row['rainfallsince'],
        amount_cumulative_historical=row['avgrainfall1971_2000'],
    )
len(df_all), m.RainFall.objects.all().count()

(4406, 4406)

In [10]:
m.RainFall.objects.all().count(), m.ReservoirState.objects.all().count()

(0, 4406)

# Reservoir Map

In [11]:
# Get the data from the database with uuid and name
reservoirs = m.Reservoir.objects.all()
reservoirs = reservoirs.values_list('name', 'uuid')
# Turn this into a dict with the name being the key
reservoirs_dict = {name: str(uuid) for name, uuid in reservoirs}
reservoirs_dict

{'la vinnuela': '0650e713-5ed4-4b9e-87a6-28b0dd4bf3ef',
 'guadalteba': '9fc5281c-13fc-41e3-8ee8-1a68f1a61244',
 'guadalhorce': '7d2e0c20-9d88-424d-9fe0-c9c67d32f436',
 'conde de guadalhorce': '6dd64514-af0e-4a72-8a64-ea8fc694e9d9',
 'la concepcion': '96ff0189-7b28-42a2-a772-0487d493af7e'}

In [20]:
filename = f'{p.cd_water}/app/frontend/src/data/reservoirs.json'
gdf = gdf_raw[gdf_raw.nombre.isin(df_matches['name_geo_full'])].copy()
gdf['name_data'] = [dict_matches[n] for n in gdf.nombre]
gdf['reservoir_uuid'] = [reservoirs_dict.get(name) for name in gdf['name_data']]
gdf.geometry = gdf.geometry.simplify(100)
gdf = gdf.to_crs("EPSG:4326")
gdf = gdf[gdf.reservoir_uuid.notnull()].copy()
gdf

,cod_emb,nombre,cod_est,cod_roea,idemb_cbrh,cod_massup,fuente_nom,area,perimetro,fecha_alta,fecha_baja,geometry,name_data,reservoir_uuid
7,8.0,Embalse del Conde de Guadalhorce,S31,n/e,4.01e+06,None,Mapa Topográfico de Andalucía 1:10000. Restitu...,4.70e+06,30445.0,2010,0,"POLYGON ((-4.83942 36.89797, -4.83882 36.89801...",conde de guadalhorce,6dd64514-af0e-4a72-8a64-ea8fc694e9d9
162,163.0,Embalse de la Concepción,S16,n/e,4.01e+06,None,Mapa Topográfico de Andalucía 1:10000. Restitu...,1.91e+06,19728.0,2010,0,"POLYGON ((-4.95956 36.57968, -4.95771 36.57969...",la concepcion,96ff0189-7b28-42a2-a772-0487d493af7e
190,191.0,Embalse de Guadalteba,S29,n/e,4.01e+06,None,Mapa Topográfico de Andalucía 1:10000. Restitu...,7.63e+06,52830.0,2010,0,"POLYGON ((-4.80203 36.94383, -4.79938 36.94079...",guadalteba,9fc5281c-13fc-41e3-8ee8-1a68f1a61244
195,196.0,Embalse de la Viñuela,S37,n/e,4.01e+06,None,Mapa Topográfico de Andalucía 1:10000. Orto 2006,3.37e+06,20598.0,2010,0,"POLYGON ((-4.17591 36.89975, -4.17475 36.89514...",la vinnuela,0650e713-5ed4-4b9e-87a6-28b0dd4bf3ef
225,226.0,Embalse de Guadalhorce,S30,n/e,4.01e+06,None,Mapa Topográfico de Andalucía 1:10000. Restitu...,6.98e+06,45492.0,2010,0,"POLYGON ((-4.80070 36.94314, -4.80229 36.94449...",guadalhorce,7d2e0c20-9d88-424d-9fe0-c9c67d32f436


In [21]:
gdf.to_file(filename, driver='GeoJSON')
gdf.iloc[0]

DriverError: Failed to create GeoJSON datasource: /Users/cg/Dropbox/code/Python/water/app/frontend/src/data/reservoirs.json: /Users/cg/Dropbox/code/Python/water/app/frontend/src/data/reservoirs.json: No such file or directory

In [22]:
filename

'/Users/cg/Dropbox/code/Python/water/app/frontend/src/data/reservoirs.json'